# Medical Image Generation with Generative AI

This comprehensive guide is designed to help you navigate the process of generating medical image on the NVIDIA DGX Cloud, focusing on leveraging the powerful capabilities of DGX systems for medical imaging applications.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/monai-cloud-api/blob/main/notebooks/Medical%20Image%20Generation%20with%20Generative%20AI.ipynb)

## Table of Contents

- Introduction
- Setup
- Generative AI Experiment Creation
- Generating Medical Image
- Conclusion

## Introduction

TODO: add introduction here

If you haven't already generated your key or if you're unsure about the process, follow our step-by-step for [Generating and Managing Your Credentials](./Generating%20and%20Managing%20Your%20Credentials.ipynb).


## Setup

In [ ]:
import requests
import json
import os

In [ ]:
# API Endpoint and Credentials
host_url = "https://api.monai.ngc.nvidia.com"
ngc_api_key = os.environ.get("MONAI_API_KEY", "<YOUR_API_KEY>")  # we recommend using environment variables for API keys, but you can also hardcode them here

# Cloud storage credentials. Needed for storing the data and results of the experiments.
access_id = "<user name for the object storage>"                 # Please fill it with the actual Access ID
access_secret = "<secret for the object storage>"                # Please fill it with the actual Access Secret

# Experiment cloud storage, where your jobs and experiments outputs will be stored.
cs_bucket = "<bucket/container name to push experiment job data to>"  # Please fill it with the actual bucket name

#### Login into NGC and API setup

In [ ]:
# Exchange NGC_API_KEY for JWT
api_url = f"{host_url}/api/v1"
response = requests.post(f"{api_url}/login", data=json.dumps({"ngc_api_key": ngc_api_key}))
assert response.status_code == 201, f"Login failed, got status code: {response.status_code}."
assert "user_id" in response.json(), "user_id is not in response."
assert "token" in response.json(), "token is not in response."
uid = response.json()["user_id"]
token = response.json()["token"]

# Construct the URL and Headers
base_url = f"{api_url}/orgs/iasixjqzw1hj"
headers = {"Authorization": f"Bearer {token}"}

## Generative AI Experiment Creation


### **1. Find the base experiment for Generative AI**

In [ ]:
endpoint = f"{base_url}/experiments"
response = requests.get(endpoint, headers=headers)
assert response.status_code == 200, f"List experiment failed, got {response.json()}."
res = response.json()
  
gen_ai_base_exps = [p for p in res if p["network_arch"] == "monai_genai" and p["name"] == "MONAI GenerativeAI"]
assert len(gen_ai_base_exps) > 0, "No base experiment found for Generative AI"
print(f"List of available base experiments for Generative AI:")
for exp in gen_ai_base_exps:
    print(f"  {exp['id']}: {exp['name']} v{exp['version']}")
base_experiment = sorted(gen_ai_base_exps, key=lambda x: x["version"])[-1]  # take the latest version
base_experiment_id = base_experiment["id"]
print("-----------------------------------------------------------------------------------------")
print(f"Base experiment ID for '{base_experiment['name']}' v{base_experiment['version']}: {base_experiment_id}")
print("-----------------------------------------------------------------------------------------")

### **2. Create Generative AI Experiment**

In [ ]:
experiment_cloud_details = {
    "cloud_type": "azure",
    "cloud_file_type": "folder",         # If the file is tar.gz key in "file", else "folder"
    "cloud_specific_details": {
        "cloud_bucket_name": cs_bucket,  # Bucket link to upload results to
        "account_name": access_id,       # Access and Secret for Azure
        "access_key": access_secret,     # Access and Secret for Azure
    }
}

data = {
    "name": "generative_ai_experiment",
    "description": "MONAI Generative AI Experiment",
    "type": "medical",
    "base_experiment": [base_experiment_id],
    "network_arch": "monai_genai",
    "cloud_details": experiment_cloud_details,
}

endpoint = f"{base_url}/experiments"
response = requests.post(endpoint, json=data, headers=headers)
assert response.status_code == 201, f"Experiment creation failed, got {response.json()}."
res = response.json()
experiment_id = res["id"]
print("Experiment creation succeeded with experiment ID:", experiment_id)
print("--------------------------------------------------------------------------------------")
print(json.dumps(res, indent=2))
print("--------------------------------------------------------------------------------------")

## Generate Medical Images

In [ ]:
data = {
    "action": "generate",
    "specs": {
        "num_output_samples": 2,
        "body_region": ["chest"],
        "organ_list": ["liver"],   
        "num_gpu": 1,
    },
}
endpoint = f"{base_url}/experiments/{experiment_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

assert response.status_code == 201, f"Create job failed, got {response.json()}."
job_id = response.json()
print("Job creation succeeded with job ID:", job_id)

## Cleaning Up

Delete the experiment after all jobs are done.

In [ ]:
endpoint = f"{base_url}/experiments/{experiment_id}/jobs/{job_id}"
response = requests.get(endpoint, headers=headers)
# If the job is not done, need to cancel it first
if response.json()["status"] != "Done":
    endpoint = f"{base_url}/experiments/{experiment_id}/jobs/{job_id}:cancel"
    response = requests.post(endpoint, headers=headers)
    assert response.status_code == 200, f"Cancel job failed, got {response.json()}."
    print(response)

endpoint = f"{base_url}/experiments/{experiment_id}"
response = requests.delete(endpoint, headers=headers)
assert response.status_code == 200, f"Delete experiment failed, got {response.json()}."
print(response)

## Conclusion

